In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from multiprocessing import Process, Queue
from datetime import date
from collections import Counter
from PyQt5 import QtCore, QtGui, QtWidgets
import time
import csv

class instagram_Crawler:
    """
        인스타그램 계정ID, 총 게시물수, 총 좋아요수, 총 댓글수, 팔로워 수, 평균 좋아요수, 평균 댓글수, 처음 올린 날짜, 마지막 올린 날짜,
        활성기간 수집 크롤러
    """
    
    def follow_Id_Crawling(self, path = '/Users/markmac/Downloads/chromedriver'): # 팔로우한 계정 크롤링
        """
            팔로우 계정 수집
            
            path: 크롬드라이버 설치 위치
        """
        self.path = path # Chrome Webdriver Path
        
        account_id = [] # 계정 ID 넣을 list
        
        driver = webdriver.Chrome(path) # Chrome webdriver path
        driver.get('https://www.instagram.com') # 사이트 주소
        
        time.sleep(1) # 1초 슬립
        driver.find_element_by_css_selector('p.izU2O > a').click() # 인스타그램 로그인 버튼 클릭
        
        time.sleep(1)
        driver.find_element_by_name('username').send_keys('hashtagcrawler@gmail.com') # 계정 ID 입력
        driver.find_element_by_name('password').send_keys('dkdlel12') # 계정 Password 입력
        
        driver.find_element_by_xpath("//button[@class = '_0mzm- sqdOP  L3NKy       ']").click() # 로그인 하기 버튼 클릭
        
        time.sleep(2) # 2초 슬립
        driver.find_element_by_xpath("//button[@class = 'aOOlW  bIiDR  ']").click() # 알림 설정 버튼 클릭
        driver.find_element_by_xpath("//span[@class = 'glyphsSpriteUser__outline__24__grey_9 u-__7']").click() # 내 정보 클릭
        
        time.sleep(1)
        driver.find_elements_by_css_selector('a.-nal3 ')[1].click() # 팔로우 버튼 클릭
        
        time.sleep(1)
        scroll = driver.find_element_by_class_name('isgrP') # 스크롤 범위
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll) # 스크롤 맨 밑으로 내리기
        
        time.sleep(1)
        # 스크롤 절반 올리기
        jsScript = """
        function move_up(element) {
            element.scrollTop = element.scrollTop - 2000;
        }

        function move_down(element) {
            console.log('Position before: ' + element.scrollTop);
            element.scrollTop = element.scrollTop + 1000;
            console.log('Position after: ' + element.scrollTop);
        }

        move_up(arguments[0]);
        """
        
        driver.execute_script(jsScript, scroll) # 스크롤 절반 올리기
        
        time.sleep(1)
        # 스크롤 다시 계속 내리기
        try:
            for i in range(10): # 스크롤 10번 내림
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll)
                time.sleep(1)
        except:
            print('error')
            
        time.sleep(1)
        html = driver.page_source # Html 소스코드 가져오기
        dom = BeautifulSoup(html, 'html.parser') # BeautifulSoup 모듈로 Html 객체구조 변환
            
        accounts_Id_List = driver.find_elements_by_xpath("//a[@class = 'FPmhX notranslate _0imsa ']") # 팔로우 계정 ID 경로
        
        accounts_Id = [j.text for j in accounts_Id_List] # 계정 ID 경로 텍스트로 리스트 삽입
            
        driver.close() # 브라우져 종료
            
        return accounts_Id
    
    def accounts_Info_Crawling(self, accounts_Id_List, output, path = '/Users/markmac/Downloads/chromedriver'):
        """
            수집한 팔로우 계정ID을 이용해 각종 데이터 수집
        """
        self.accounts_Id_List = accounts_Id_List
        self.output = output
        self.path = path
        
        accounts_Id = [] # 계정ID
        post_Counts = [] # 게시물 수
        follower_Counts = [] # 팔로워 수
        last_Activity_Date = [] # 최근 활동 날짜
        first_Activity_Date = [] # 처음 활동 날짜
        active_Period = [] # 계정 활성 기간
        good_Counts = [] # 총 좋아요 수
        avg_Goods = [] # 평균 좋아요 수
        hashtags_Counts = [] # 제일 큰 해시태그/빈도 수
        
        for i in range(len(accounts_Id_List)):
            hashtags = [] # 해시태그 html 소스코드 저장
            words = [] # 해시태그와 계정태그 저장
            hashtags_data = [] # 해시태그 저장
            good = 0
            
            driver = webdriver.Chrome(path)
            driver.get('https://www.instagram.com/' + accounts_Id_List[i])
            
            time.sleep(1)
            html = driver.page_source
            dom = BeautifulSoup(html, 'html.parser')
            
            accounts_Id.append(dom.select('h1._7UhW9.fKFbl.yUEEX.KV-D4.fDxYl')[0].text) # 계정ID 저장
            post_Counts.append(int(dom.select('span.g47SY')[0].text.replace(',', ''))) # 게시물 수 저장
            follower_Counts.append(int(dom.select('span.g47SY')[1]['title'].replace(',', ''))) # 팔로워 수 저장
            
            driver.find_element_by_class_name('_9AhH0').click() # 첫번째 게시물 클릭
            
            while True:
                try:
                    time.sleep(1)
                    html = driver.page_source
                    dom = BeautifulSoup(html, 'html.parser')
                    
                    # 첫 게시물이 동영상 게시물일때
                    if dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') == [] and good == 0:
                        time.sleep(1)
                        last_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10]) # 최근 활동 날짜 수집

                        last_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0]) # 최근 활동 년도
                        last_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1]) # 최근 활동 월
                        last_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2]) # 최근 활동 일

                        hashtags += dom.select('div.C4VMK > span > a') # 해시태그 소스코드 저장

                        driver.find_element_by_class_name('vcOH2').click() # 조회수 클릭

                        time.sleep(1)
                        html = driver.page_source
                        dom = BeautifulSoup(html, 'html.parser')

                        good = int(dom.select('div.vJRqr > span')[0].text.replace(',', '')) # 좋아요 갯수 저장

                        driver.find_element_by_class_name('QhbhU').click() # 게시물 클릭 후 나감

                        # 다음 게시물이 없을 때
                        if dom.select('a.HBoOv.coreSpriteRightPaginationArrow') == []:
                            time.sleep(1)
                            html = driver.page_source
                            dom = BeautifulSoup(html, 'html.parser')

                            first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10]) # 처음 활동 날짜 수집

                            first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0]) # 첫 활동 년도
                            first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1]) # 첫 활동 월
                            first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2]) # 첫 활동 일

                            last = date(last_Year, last_Month, last_Day) # 최근활동날짜
                            first = date(first_Year, first_Month, first_Day) # 처음활동날짜

                            period = last - first # 활성기간
                            active_Period.append(period.days) # 활성기간 저장

                            good_Counts.append(good) # 총 좋아요 수 저장
                            avg_Goods.append(round(good / post_Counts[0])) # 평균 좋아요 수 저장

                            words = [j.text for j in hashtags] # 해시태그 + 계정태그 저장
                            hashtags_data = [x for x in words if x[0] == '#'] # 순수 해시태그 추출
                            hashtags_Counts.append(Counter(hashtags_data).most_common()[:1]) # 제일 빈도가 큰 해시태그 저장

                            driver.close() # 브라우저 닫기
                            break

                        # 다음 게시물이 있으면
                        else:
                            time.sleep(1)
                            # 다음 버튼 클릭
                            driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()

                    # 댓글, 좋아요 수가 0이 아닐때 영상 게시물
                    elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') == [] and good != 0:
                        time.sleep(1)
                        hashtags += dom.select('div.C4VMK > span > a')

                        driver.find_element_by_class_name('vcOH2').click()

                        time.sleep(1)
                        html = driver.page_source
                        dom = BeautifulSoup(html, 'html.parser')

                        good += int(dom.select('div.vJRqr > span')[0].text.replace(',', ''))

                        driver.find_element_by_class_name('QhbhU').click()

                        if dom.select('a.HBoOv.coreSpriteRightPaginationArrow') == []:
                            time.sleep(1)
                            html = driver.page_source
                            dom = BeautifulSoup(html, 'html.parser')

                            first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                            first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                            first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                            first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                            last = date(last_Year, last_Month, last_Day)
                            first = date(first_Year, first_Month, first_Day)

                            period = last - first
                            active_Period.append(period.days)

                            good_Counts.append(good)
                            avg_Goods.append(round(good / post_Counts[0]))

                            words = [j.text for j in hashtags]
                            hashtags_data = [x for x in words if x[0] == '#']
                            hashtags_Counts.append(Counter(hashtags_data).most_common()[:1])

                            driver.close()
                            break

                        else:
                            time.sleep(1)             
                            driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()

                    # 첫 사진 게시물일때
                    elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') != [] and good == 0:
                        time.sleep(1)
                        last_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                        last_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                        last_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                        last_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                        hashtags += dom.select('div.C4VMK > span > a')
                        good = int(dom.select('button._0mzm-.sqdOP.yWX7d._8A5w5 > span')[0].text.replace(',', ''))

                        if dom.select('a.HBoOv.coreSpriteRightPaginationArrow') == []:    
                            time.sleep(1)
                            first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                            first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                            first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                            first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                            last = date(last_Year, last_Month, last_Day)
                            first = date(first_Year, first_Month, first_Day)

                            period = last - first
                            active_Period.append(period.days)
                            
                            good_Counts.append(good)
                            avg_Goods.append(round(good / post_Counts[0]))

                            words = [j.text for j in hashtags]
                            hashtags_data = [x for x in words if x[0] == '#']
                            hashtags_Counts.append(Counter(hashtags_data).most_common()[:1])

                            driver.close()
                            break

                        else:
                            time.sleep(1)
                            driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()

                    # 둘다 0일때 사진 게시물
                    else:
                        time.sleep(1)
                        hashtags += dom.select('div.C4VMK > span > a')
                        good += int(dom.select('button._0mzm-.sqdOP.yWX7d._8A5w5 > span')[0].text.replace(',', ''))

                        if dom.select('a.HBoOv.coreSpriteRightPaginationArrow') == []:    
                            time.sleep(1)
                            first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                            first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                            first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                            first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                            last = date(last_Year, last_Month, last_Day)
                            first = date(first_Year, first_Month, first_Day)

                            period = last - first
                            active_Period.append(period.days)
                            
                            good_Counts.append(good)
                            avg_Goods.append(round(good / post_Counts[0]))

                            words = [j.text for j in hashtags]
                            hashtags_data = [x for x in words if x[0] == '#']
                            hashtags_Counts.append(Counter(hashtags_data).most_common()[:1])

                            driver.close()
                            break

                        else:
                            time.sleep(1)
                            driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                                
                except:
                    break
                
        total_data = {
            '계정ID':accounts_Id, '게시물수':post_Counts, '팔로워수':follower_Counts, '평균좋아요수':avg_Goods, 
            '총좋아요수':good_Counts, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, 
            '계정활성기간':active_Period, '제일많이나온해시태그':hashtags_Counts
        }
        
        output.put(total_data)
        
    def multiprocessing_Crawling(self, accounts_Id_List):
        self.accounts_Id_List = accounts_Id_List
        
        accounts_Id_List_1 = []
        accounts_Id_List_2 = []
        accounts_Id_List_3 = []
        accounts_Id_List_4 = []
        accounts_Id_List_5 = []
        accounts_Id_List_6 = []
        instagram_Data = []
        procs = []
                
        # multiprocessing 모듈을 사용하기 위해 수집한 계정데이터 분리
        accounts_Id_List_1 += accounts_Id_List[:0]
        accounts_Id_List_2 += accounts_Id_List[0:1]
        accounts_Id_List_3 += accounts_Id_List[3:5]
        accounts_Id_List_4 += accounts_Id_List[5:7]
        accounts_Id_List_5 += accounts_Id_List[7:9]
        accounts_Id_List_6 += accounts_Id_List[47:57]
        
        output1 = Queue()
        output2 = Queue()
        output3 = Queue()
        output4 = Queue()
        output5 = Queue()
        output6 = Queue()

        procs.append(Process(target = instagram.accounts_Info_Crawling, args = (accounts_Id_List_1, output1)))
        procs.append(Process(target = instagram.accounts_Info_Crawling, args = (accounts_Id_List_2, output2)))
        procs.append(Process(target = instagram.accounts_Info_Crawling, args = (accounts_Id_List_3, output3)))
        procs.append(Process(target = instagram.accounts_Info_Crawling, args = (accounts_Id_List_4, output4)))
        procs.append(Process(target = instagram.accounts_Info_Crawling, args = (accounts_Id_List_5, output5)))
        procs.append(Process(target = instagram.accounts_Info_Crawling, args = (accounts_Id_List_6, output6)))

        for p in procs:
            p.start()

        instagram_Data += output1.get()
        instagram_Data += output2.get()
        instagram_Data += output3.get()
        instagram_Data += output4.get()
        instagram_Data += output5.get()
        instagram_Data += output6.get()

        for p in procs:
            p.join()
            
        return instagram_Data
    
    def save_csv(self, dict_data):
        self.dict_data = dict_data
        
        data = []
        
        with open('test_file.csv', 'w') as f:
            w = csv.DictWriter(f, dict_data.keys())
            w.writeheader()
            for i in range(len(dict_data['계정ID'])):
                data.append({'계정ID' : dict_data['계정ID'][i], '게시물수' : dict_data['게시물수'][i], 
                             '팔로워수' : dict_data['팔로워수'][i], '평균좋아요수' : dict_data['평균좋아요수'][i], 
                             '총좋아요수' : dict_data['총좋아요수'][i], '최근게시물날짜' : dict_data['최근게시물날짜'][i], 
                             '처음게시물날짜' : dict_data['처음게시물날짜'][i], '계정활성기간' : dict_data['계정활성기간'][i],
                             '제일많이나온해시태그' : dict_data['제일많이나온해시태그'][i]})
            w.writerows(data)

In [2]:
instagram = instagram_Crawler()

In [3]:
accounts_Id = instagram.follow_Id_Crawling()

In [4]:
accounts_Id

['jayflowsik',
 'killagramz3xl',
 'chachamalone',
 'boycold_',
 'uglyduck062',
 'wavycolde',
 'woogie_park',
 'longlivesmdc',
 'anythinggoes85',
 'coogie0123',
 'hwimmm',
 'yunbnotlilb',
 'supergreathurricane',
 'drunkentigerjk',
 'g2slife',
 'gaekogeem',
 'nucksal',
 'penomadeincorea',
 'keemhongwu',
 'huckleberryp84',
 'deepflow39',
 'superstarjimmyfuckedup',
 'originalgimchi',
 'hiolnl',
 'dprlive',
 'crush9244',
 'satgotloco',
 'junoflo',
 'woodie_gochild',
 'younghotyellow94',
 'code_kunst',
 'bewhy.meshasoulja',
 'nasungcityboy',
 'wejustmusic',
 'thisloop',
 'jparkitrighthere',
 'hashblanccoa',
 'callmegray',
 'aomgofficial',
 'ph1boyyy',
 'paloaltongue',
 'woozico0914',
 'thequiett',
 'munchinthepool',
 'dok2gonzo',
 'noahmik',
 'realllllmino',
 'kiff_websterb',
 'changmo_',
 'itsjustswings',
 'kidcozyboy',
 'noelcozyboy',
 'yohanheaven',
 'osshun_gum',
 'jvckiwai',
 'hankyungsoo',
 'giriboy91']

In [3]:
aa = ['boycold_', 'longlivesmdc', 'coogie0123', 'yunbnotlilb', 'drunkentigerjk', 'nucksal', 'superstarjimmyfuckedup', 'originalgimchi', 'hiolnl', 'woodie_gochild']

In [4]:
aa

['boycold_',
 'longlivesmdc',
 'coogie0123',
 'yunbnotlilb',
 'drunkentigerjk',
 'nucksal',
 'superstarjimmyfuckedup',
 'originalgimchi',
 'hiolnl',
 'woodie_gochild']

In [5]:
instagram_Data = instagram.accounts_Info_Crawling(aa)

In [4]:
instagram_Data = instagram.multiprocessing_Crawling(aa)

In [6]:
instagram.save_csv(instagram_Data)